# Running `fftvis` with the GPU Backend

In this notebook, we'll give a brief demonstration of how to use `fftvis` with GPU acceleration to simulate visibilities for a set of input simulation parameters. This tutorial is very similar to the one included in the `matvis` simulator package. One should consolunt that tutorial for a difference in the use cases of `fftvis` and `matvis`.

<div class="alert alert-info">

__Note__

The absolute easiest way to use `fftvis` is via the `hera_sim` [plugin interface](https://hera-sim.readthedocs.io/en/latest/tutorials/hera_sim_vis_cli.html).
</div>

<div class="alert alert-warning">

__Warning__

Before running this tutorial, you should make sure you understand the basic concepts and algorithm that `fftvis` uses. You can read up on that here.
</div>

In [1]:
# Standard imports
import numpy as np
import healpy as hp
from astropy.time import Time
import matplotlib.pyplot as plt

# HERA-stack imports
import fftvis
import matvis
from hera_sim.antpos import hex_array
from pyuvdata.telescopes import Telescope
from pyuvdata.analytic_beam import AiryBeam

/home/kartikmandar/anaconda3/envs/matvis-env/lib/python3.11/site-packages/pyuvdata/analytic_beam.py:111: UserWarning: basis_vector_type was not defined, defaulting to azimuth and zenith_angle.
  warnings.warn(


## Setup Telescope / Observation Parameters

We need a few input parameters to setup our observation: antenna positions, beam models, and a sky model.

Here we will set up a very simple observation for introductory purposes.

First, create our antenna positions. We define this as a dictionary, which maps an antenna number to its 3D East-North-Up position relative to the array centre. Here, we define just a hexagonal array of 15 antennas using `hera_sim.antpos.hex_array`.

In [2]:
# define antenna array positions
antpos = hex_array(3, split_core=True, outriggers=0)

Next, we define the beam to be used by all antennas in the array. Unlike `matvis` and `pyuvsim`, `fftvis` currently restricts users to a single beam for all antennas. The specified beam must be a `UVBeam` or `AnalyticBeam` object from `pyuvdata`. Alternatively, you can create a custom `AnalyticBeam` class (see the pyuvdata tutorial on `UVBeam` objects for guidance). For this simulation, we will use a simple, frequency-dependent Airy beam corresponding to a dish size of 14 meters.

**Note for GPU backend**: The GPU backend requires UVBeam objects for beam evaluation. We'll convert our AnalyticBeam to a UVBeam for GPU compatibility while maintaining the same beam parameters.

In [3]:
# define antenna beam using pyuvdata.analytic_beam.AiryBeam with a dish size of 14 meters
analytic_beam = AiryBeam(diameter=14.0)

# For GPU compatibility, convert AnalyticBeam to UVBeam
# We'll define the frequency range and angular resolution for the conversion
freq_array = np.linspace(100e6, 120e6, 20)  # Same as our observation frequencies
naz, nza = 360, 180  # Angular resolution
beam = analytic_beam.to_uvbeam(
    freq_array=freq_array,
    axis1_array=np.linspace(0, 2 * np.pi, naz + 1)[:-1],  # azimuth
    axis2_array=np.linspace(0, np.pi, nza + 1),  # zenith angle
)

We also required to provide `fftvis` with the observational configuration including a frequency array, a time array, and a telescope location. The frequency array specifies the observation frequencies in units of Hz. The time array defines the observation times using an `astropy.time.Time` object, with times specified in Julian Dates and configured with the appropriate format and scale. The telescope location specifies the geographic position of the array and can be defined either using `astropy.coordinates.EarthLocation` with a known site name or through `pyuvdata.telescopes.Telescope` by selecting a predefined telescope location supported within `pyuvdata`.

In [4]:
# define a list of frequencies in units of Hz
nfreqs = 20
freqs = np.linspace(100e6, 120e6, nfreqs)

In [5]:
# define a list of times with an astropy time.Time object
ntimes = 30
times = Time(np.linspace(2459845, 2459845.05, ntimes), format='jd', scale='utc')

In [6]:
from astropy.coordinates import EarthLocation

# define using astropy.coordinates.EarthLocation
telescope_loc = EarthLocation.of_site('meerkat')

# define the telescope location using the pyuvdata.telescopes.Telescope
telescope_loc = Telescope.from_known_telescopes('hera').location

## Setup Sky Model

Like `matvis`, `fftvis` makes the point source approximation -- that is it makes breaks a continuous sky model into a discrete number of point sources that it sums over when computing the visibilities. In this notebook, we'll assume the point source approximation by discretizing the sky with a randomly generated HEALpix map.

In [7]:
# number of sources
nside = 64
nsource = hp.nside2npix(nside)

# pixels can be defined as point sources randomly distributed over the full sky
ra = np.deg2rad(np.random.uniform(0, 360, nsource))        # ra of each source (in rad)
dec = np.deg2rad(np.random.uniform(-90, 90.0, nsource))    # dec of each source (in rad)

# define sky model using healpix map
dec, ra = hp.pix2ang(nside, np.arange(nsource))
dec -= np.pi / 2

# define the flux of the sources as a function of frequency. Here, we define smooth spectrum sources
flux = np.random.uniform(0, 1, nsource)                              # flux of each source at 100MHz (in Jy)
alpha = np.ones(nsource) * -0.8                      # sp. index of each source

# Now get the (Nsource, Nfreq) array of the flux of each source at each frequency.
flux_allfreq = ((freqs[:, np.newaxis] / freqs[0]) ** alpha.T * flux.T).T

## Run `fftvis` in single processor mode

Now that we've setup all our parameters, we can easily run the simulation using the high-level wrapper API. Along with the configuration we've already defined, the `fftvis.simulate.simulate_vis` wrapper takes a few extra options. One of the most important is `polarized`: if true, then full polarized visibilities are returned (with shape `(nfreqs, ntimes, nfeed, nfeed, nbls)`), otherwise, unpolarized visibilities are returned (with shape (nfreqs, ntimes, nants, nants)). In our case, the `AnalyticBeam` objects we're using don't support polarization, so we set this to false.

We can also set the `precision` parameter, which switches between 32-bit (if `precision=1`) and 64-bit (if `precision=2`) floating precision. In addition to controlling the floating point precision, the `fftvis` also includes a parameter `eps` which can improve the precision of the underlying Non-Uniform FFT. The runtime of the simulation is roughly inversely proportional to the `eps` parameter for a set `precision` value.

In [8]:
# Define subset of baselines we're interested in for simulating
baselines = [(i, j) for i in range(len(antpos)) for j in range(len(antpos))]

In [10]:
%%time
# simulate visibilities with the new API
vis_vc = fftvis.simulate_vis(
    ants=antpos,
    fluxes=flux_allfreq,
    ra=ra,
    dec=dec,
    freqs=freqs,
    times=times.jd,
    telescope_loc=telescope_loc,
    beam=beam,
    polarized=False,
    precision=2,
    nprocesses=1,
    baselines=baselines,
    backend="gpu"  # Use GPU backend instead of CPU
)

ValueError: operands could not be broadcast together with shapes (24576,) (24576, 20) (24576,)

## Compare GPU and CPU Backends

Let's run the same simulation with the CPU backend for comparison. We expect the results to be nearly identical, with the main difference being the computation time.

In [ ]:
%%time
# Simulate visibilities using the CPU backend
vis_cpu = fftvis.simulate_vis(
    ants=antpos,
    fluxes=flux_allfreq,
    ra=ra,
    dec=dec,
    freqs=freqs,
    times=times.jd,
    telescope_loc=telescope_loc,
    beam=beam,
    polarized=False,
    precision=2,
    nprocesses=1,
    baselines=baselines,
    backend="cpu"  # Use CPU backend
)

In [ ]:
# Check that results from GPU and CPU are equivalent
if gpu_available:
    # The results should be very close but not exactly the same due to floating-point differences
    print(f"Maximum absolute difference: {np.max(np.abs(vis_gpu - vis_cpu))}")
    print(f"Are GPU and CPU results close? {np.allclose(vis_gpu, vis_cpu, rtol=1e-5, atol=1e-7)}")

## Benchmark Performance: GPU vs CPU

Let's benchmark the performance difference between GPU and CPU backends with increasing number of sources. The GPU advantage typically becomes more apparent with larger datasets.

In [ ]:
def benchmark_performance(nsides, ntimes=10, nfreqs=5):
    """Benchmark GPU vs CPU performance for different HEALPix nsides."""
    
    # Shorter time and frequency arrays for benchmarking
    short_freqs = np.linspace(100e6, 120e6, nfreqs)
    short_times = Time(np.linspace(2459845, 2459845.02, ntimes), format='jd', scale='utc')
    
    results = []
    
    for nside in nsides:
        nsource = hp.nside2npix(nside)
        print(f"Running benchmark with nside={nside}, nsource={nsource}")
        
        # Create sky model
        dec, ra = hp.pix2ang(nside, np.arange(nsource))
        dec -= np.pi / 2
        flux = np.random.uniform(0, 1, nsource)
        alpha = np.ones(nsource) * -0.8
        flux_allfreq = ((short_freqs[:, np.newaxis] / short_freqs[0]) ** alpha.T * flux.T).T
        
        # Time CPU simulation
        t0 = time.time()
        _ = fftvis.simulate_vis(
            ants=antpos,
            fluxes=flux_allfreq,
            ra=ra,
            dec=dec,
            freqs=short_freqs,
            times=short_times.jd,
            telescope_loc=telescope_loc,
            beam=beam,
            polarized=False,
            precision=2,
            nprocesses=1,
            baselines=baselines[:10],  # Use fewer baselines for speed
            backend="cpu"
        )
        cpu_time = time.time() - t0
        
        # Time GPU simulation
        gpu_time = None
        if gpu_available:
            t0 = time.time()
            _ = fftvis.simulate_vis(
                ants=antpos,
                fluxes=flux_allfreq,
                ra=ra,
                dec=dec,
                freqs=short_freqs,
                times=short_times.jd,
                telescope_loc=telescope_loc,
                beam=beam,
                polarized=False,
                precision=2,
                nprocesses=1,
                baselines=baselines[:10],  # Use fewer baselines for speed
                backend="gpu"
            )
            gpu_time = time.time() - t0
            # Clear GPU memory
            if 'cp' in globals():
                cp.cuda.runtime.deviceSynchronize()
                cp.get_default_memory_pool().free_all_blocks()
        
        results.append((nside, nsource, cpu_time, gpu_time))
    
    return results

In [ ]:
# Run benchmarks with increasing HEALPix nside values
# Skip if GPU is not available
if gpu_available:
    benchmark_results = benchmark_performance([8, 16, 32, 64])
else:
    print("GPU not available. Skipping benchmarks.")

In [ ]:
# Plot the benchmark results
if gpu_available and 'benchmark_results' in locals():
    nsides, nsources, cpu_times, gpu_times = zip(*benchmark_results)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Plot execution times
    ax1.plot(nsources, cpu_times, 'o-', label='CPU')
    ax1.plot(nsources, gpu_times, 's-', label='GPU')
    ax1.set_xlabel('Number of Sources')
    ax1.set_ylabel('Execution Time (s)')
    ax1.set_title('Execution Time Comparison')
    ax1.legend()
    ax1.grid(True)
    
    # Plot speedup ratios
    speedups = [cpu/gpu for cpu, gpu in zip(cpu_times, gpu_times)]
    ax2.plot(nsources, speedups, 'o-')
    ax2.set_xlabel('Number of Sources')
    ax2.set_ylabel('Speedup (CPU time / GPU time)')
    ax2.set_title('GPU Speedup Factor')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

## Plot Visibility Results

We'll plot the visibility amplitude and phase from the GPU simulation, similar to what we did in the CPU tutorial.

In [ ]:
# Use GPU results if available, otherwise CPU results
vis_to_plot = vis_gpu if gpu_available else vis_cpu

fig, axs = plt.subplots(1, 2, figsize=(10, 6))
for bl_index, bl in enumerate(baselines[:3]):
    axs[0].semilogy(freqs / 1e6, np.abs(vis_to_plot[:, 0, bl_index]))
    axs[1].plot(freqs / 1e6, np.angle(vis_to_plot[:, 0, bl_index]), label=f"b = {bl[0]}")

axs[1].legend()
axs[0].set_xlabel('Frequency [MHz]')
axs[1].set_xlabel('Frequency [MHz]')
axs[0].set_ylabel('Amplitude [Jy]')
axs[1].set_ylabel('Phase [rad]')
axs[1].set_ylim(-np.pi * 1.1, np.pi * 1.1)
axs[0].grid()
axs[1].grid()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 6))
for bl_index, bl in enumerate(baselines[:3]):
    axs[0].semilogy(times.unix - times.unix[0], np.abs(vis_to_plot[0, :, bl_index]))
    axs[1].plot(times.unix - times.unix[0], np.angle(vis_to_plot[0, :, bl_index]), label=f"b = {bl[0]}")

axs[0].set_xlabel('Times [s]')
axs[1].set_xlabel('Times [s]')
axs[0].set_ylabel('Amplitude [Jy]')
axs[1].set_ylabel('Phase [rad]')
axs[1].set_ylim(-np.pi * 1.1, np.pi * 1.1)
axs[0].grid()
axs[1].grid()
plt.legend()
plt.show()

## Conclusion

The GPU backend of `fftvis` provides a significant speedup compared to the CPU backend, especially for larger sky models with many sources. The main advantages are:

1. Accelerated non-uniform FFT operations using `cufinufft`
2. Parallel processing of source computations on the GPU
3. Efficient beam interpolation using GPU-accelerated map coordinates

When working with large simulations, the GPU backend is recommended if suitable hardware is available. For smaller simulations, the overhead of data transfer between CPU and GPU might reduce the performance advantage.

In [ ]:
# Clean up GPU memory if we used it
if gpu_available and 'cp' in globals():
    cp.cuda.runtime.deviceSynchronize()
    cp.get_default_memory_pool().free_all_blocks()